In [1]:
from Metrotv import Metrotv #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [7]:
obj = Metrotv()
end_date = date(2018, 8, 8)
start_date = date(2018, 1, 23)
init_page = 1
list_cat = [['internasional', 'internasional'], ['otomotif', 'otomotif'], ['ekonomi', 'ekonomi'],['bola','bola'],
            ['olahraga','olahraga'],['teknologi','teknologi'],['hiburan','hiburan'],['rona','rona']]
init_cat = list_cat[1:]

In [8]:
for single_date in daterange(start_date, end_date):
    for cat in init_cat:
        clear_output()
        print(single_date.strftime("%Y-%m-%d"))
        print(cat[0])
        obj.getAllBerita([], init_page, 0, cat[0], cat[1], single_date.strftime("%Y/%m/%d"))
        init_cat = list_cat
    init_page = 1

2018-01-23
bola
page  1
http://bola.metrotvnews.com/index/2018/01/23/0
http://bola.metrotvnews.com/indonesia/MkMML8Vk-psm-berencana-kembali-gelar-tc-di-bali
Insert berita  PSM Berencana Kembali Gelar TC di Bali
masuk
page  2
http://bola.metrotvnews.com/index/2018/01/23/30


KeyboardInterrupt: 